In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Activation, Conv2, Dense, Dropout
from tensorflow.keras.optimizers import Adagrad, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub

import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy

from PIL import Image

# ラベル付け(0春 1夏 2冬)
from label import Label, SQUARE_LENGTH


# 画像を置いてるパス
image_path = "../image/eye_dlib"

In [2]:
# 画像の読み込み
datagen = ImageDataGenerator(
    rescale=1./255
#     validation_split=0.1
    )

train_generator = datagen.flow_from_directory(
    image_path + "/train",
    target_size=(SQUARE_LENGTH, SQUARE_LENGTH),
    class_mode='binary',
    batch_size=32,
    subset='training',
)

Found 17206 images belonging to 2 classes.


In [ ]:
print("start")

# モデルの構築
model = keras.Sequential([
#     keras.layers.Conv2D(32, (8,8), activation="relu"),
#     keras.layers.Conv2D(32, (8,8), activation="relu"),
#     keras.layers.MaxPooling2D(pool_size=(4,4)),
#     keras.layers.Flatten(),
#     keras.layers.Dropout(0.2),
#     keras.layers.Dense(200, activation="relu"),
#     keras.layers.Dropout(0.4),
#     keras.layers.Dense(200, activation="relu"),
#     keras.layers.Dropout(0.4),
#     keras.layers.Dense(1),
#     keras.layers.Activation("softmax")
    hub.KerasLayer(
        "https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1",
        trainable=False,
    ),
    keras.layers.Dense(1, activation="sigmoid")
])
# memo: shapeで警告出るのは、keras.layers.reshape((...)),imput_shape=(...))を使えば良さそう

# # モデルをコンパイル
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])
# 学習を実行
model.fit(train_generator, steps_per_epoch=1, epochs=30)
print(model.summary())

start


In [3]:
# テスト用ディレクトリの画像でチェック。正解率を表示する。
total = 0.
ok_count = 0.

# あとで消す
# 学習済みモデル
MODEL_NAME = "./model.h5"

# 学習済みモデルの読み込み
model = models.load_model(MODEL_NAME, custom_objects={"KerasLayer": hub.KerasLayer})

# test_path = image_path + "/test"
test_path = "D:/python/app_trim"
for dir in os.listdir(test_path):
    if dir == ".DS_Store":
        continue

    dir_path = test_path + "/" + dir
    label = Label[dir].value

    for file in os.listdir(dir_path):
        if file != ".DS_Store":
            file_path = dir_path + "/" + file
            image = np.array(Image.open(file_path).resize(size=(SQUARE_LENGTH, SQUARE_LENGTH)))
#             image = image.transpose(2, 0, 1)
#             image = image.reshape(image.shape[0] * image.shape[1] * image.shape[2], 1)
            result = model.predict(np.array([image / 255.]))
            print("[" + str(int(total)) + "]label:", label, "result:", result)

            total += 1.

            if abs(label - result) < 0.5:
                ok_count += 1.

print("seikai: ", ok_count / total * 100, "%")

[0]label: 0 result: [[0.6749223]]
[1]label: 0 result: [[0.69699407]]
[2]label: 0 result: [[0.6565488]]
[3]label: 0 result: [[0.5871938]]
[4]label: 0 result: [[0.5205791]]
[5]label: 0 result: [[0.6707914]]
[6]label: 0 result: [[0.66173184]]
[7]label: 0 result: [[0.5684778]]
[8]label: 0 result: [[0.54242045]]
[9]label: 0 result: [[0.56782067]]
[10]label: 0 result: [[0.48480222]]
[11]label: 0 result: [[0.6151486]]
[12]label: 0 result: [[0.68694156]]
[13]label: 0 result: [[0.70879275]]
[14]label: 0 result: [[0.6513451]]
[15]label: 0 result: [[0.5693022]]
[16]label: 0 result: [[0.67531043]]
[17]label: 0 result: [[0.48242638]]
[18]label: 0 result: [[0.65325737]]
[19]label: 0 result: [[0.48922375]]
[20]label: 0 result: [[0.38711995]]
seikai:  19.047619047619047 %


In [13]:
#tf.saved_model.simple_save(sess, './models', inputs={"input": x}, outputs={"output": out})
# model.saveでいいらしい

AttributeError: module 'tensorflow._api.v2.saved_model' has no attribute 'simple_save'

In [14]:
model.save(image_path + "/model.h5")